In [36]:
import numpy as np
import pandas as pd

import scipy.stats as st

import astropy.units as u
from astropy.coordinates import SkyCoord

import plotly.graph_objects as go

In [77]:
datas = [pd.read_fwf(f'/content/{i}_angulos_solares.txt') for i in range(10)]

for i in range(10):

  if 'AM' in datas[i].columns:

    datas[i] = datas[i].drop(columns = 'AM')

  if len(datas[i].columns) == 1:

    datas[i] = datas[i].iloc[:, 0].str.split(' ', expand=True)


  datas[i].columns = ['Time', 'Angle']
  datas[i].Angle = datas[i].Angle.astype(float)

  datas[i].Time = pd.to_timedelta(datas[i].Time + ':00').dt.total_seconds()/3600
  datas[i] = datas[i].groupby('Time')['Angle'].mean().reset_index()

In [110]:
w = np.array([])
for k in range(10):
  a = datas[k]

  omegas = np.array([[abs((a.Angle[j] - a.Angle[i])/(a.Time[j] - a.Time[i]))
                    if i != j else 0
                    for j in range(a.shape[0])] for i in range(a.shape[0])]
                    )

  omegas = omegas[omegas != 0].flatten()

  w = np.append(w, omegas)

  omegas_tlc = np.mean(np.random.choice(omegas, (100, 100), replace = True), axis=1)

  print((omegas.mean(), *st.ttest_1samp(omegas_tlc, popmean = w_tlc.mean(), alternative = 'two-sided', nan_policy = 'omit')))

(13.117918069584734, 1.2960069382606367, 0.19798638844254332)
(13.117918069584734, 1.7531217153742726, 0.08267651684002306)
(12.67412428072445, -11.979808132990073, 5.666797199858084e-21)
(12.67412428072445, -9.897625348465983, 1.8295587844185298e-16)
(13.487637704607405, 16.076880028638996, 2.3476187900564324e-29)
(13.487637704607405, 18.649339632506653, 3.627815962059891e-34)
(14.319227513227498, 319.90956864367956, 4.66329000228137e-151)
(14.319227513227498, 301.92824194409394, 1.4236238798199162e-148)
(12.374700627456601, -15.71661752073065, 1.1842159078750503e-28)
(12.374700627456601, -19.985473588252578, 1.596072126615932e-36)


In [138]:
w_tlc.std(ddof = 1)

0.1422529345346981

In [129]:
w_tlc =  np.mean(np.random.choice(w, (500, 500), replace = True), axis=1)

In [136]:
st.ttest_1samp(w_tlc, popmean = w_tlc.mean(), alternative = 'two-sided', nan_policy = 'omit').confidence_interval(0.99)

ConfidenceInterval(low=13.088479564480139, high=13.121378915662845)

In [94]:
st.ttest_1samp(w_tlc, popmean = 15*np.sin(80*u.deg), alternative = 'two-sided', nan_policy = 'omit').confidence_interval(0.6)

ConfidenceInterval(low=13.10097119693603, high=13.10846350981602)

In [96]:
st.ttest_1samp(w_tlc, popmean = 15*np.sin(80*u.deg), alternative = 'two-sided', nan_policy = 'omit').confidence_interval(0.9)

ConfidenceInterval(low=13.097392251029728, high=13.112042455722321)

In [95]:
st.ttest_1samp(w_tlc, popmean = 15*np.sin(80*u.deg), alternative = 'two-sided', nan_policy = 'omit').confidence_interval(0.99)

ConfidenceInterval(low=13.093234995410151, high=13.116199711341897)

In [127]:
st.skew(x), st.kurtosis(x)

(-0.6547070289359906, 2.534505748901415)

In [125]:
ancho = 3*(w-w_tlc.mean()).std()/w.shape[0]**(1/3)

go.Figure(data = [go.Histogram(x = w - w_tlc.mean(),
                               histnorm = 'probability density',
                               xbins_size = ancho)],

          layout = go.Layout(height = 600, width = 900,
                             xaxis_title = 'Angular Velocity Bias [deg/h]',
                             yaxis_title = 'Probability Density'))

In [87]:
ancho = 3*w_tlc.std()/w_tlc.shape[0]**(1/3)

go.Figure(data = [go.Histogram(x = w_tlc,
                               histnorm = 'probability density',
                               xbins_size = ancho)],

          layout = go.Layout(height = 600, width = 900,
                             title_text = "Mean Solar Angular Velocity",
                             title_x = 0.5, title_y = 0.93,
                             xaxis_title = 'Mean Angular Velocity [deg/h]',
                             yaxis_title = 'Probability Density'))

In [29]:
ancho = 3*omegas.std()/omegas.shape[0]**(1/3)

go.Figure(data = [go.Histogram(x = omegas,
                               histnorm = 'probability density',
                               xbins_size = ancho)],

          layout = go.Layout(height = 600, width = 900,
                             title_text = f"Solar Angular Velocity<br>(ω = {omegas.mean():.2f} deg/h)",
                             title_x = 0.5, title_y = 0.93,
                             xaxis_title = 'Angular Velocity [deg/h]',
                             yaxis_title = 'Probability Density'))

In [4]:
# @title
phi = 6.2320*u.deg
epsilon = 23.4364*u.deg
omega_0 = 2*np.pi/365.2422 * u.rad
lambda_0 = 151.797*u.deg

for i in range(10):

  datas[i]['dec'] = np.arcsin(np.sin(epsilon)*np.sin(lambda_0 + omega_0*datas[i].Time/24))

  datas[i]['H'] = np.arccos((np.sin(datas[i].Angle*u.deg.to(u.rad)) - np.sin(datas[i].dec)*np.sin(phi))/\
                   (np.cos(datas[i].dec)*np.cos(phi)))

  coords = SkyCoord(ra = datas[i].H, dec = datas[i].dec, unit = u.rad)

  datas[i]['w'] = coords[:-1].separation(coords[1:]).deg/datas[i].Time.diff()[1:]

In [58]:
go.Figure(data = [go.Scatter(x = data.Time,
                             y = data.Angle,
                             mode = 'markers',
                             showlegend = False) for data in datas],

          layout = go.Layout(height = 600, width = 900,
                             title_text = "Daily Motion in Solar Altitude",
                             title_x = 0.5, title_y = 0.93,
                             xaxis_title = 'Time [h]',
                             yaxis_title = 'Alt [deg]'))



---

#**Parte 2**

In [34]:
df = pd.concat(datas)